In [1]:
# https://drive.google.com/file/d/1cOjcgQ8tZl6tOe9FQb05ZCFnUXy1MKwJ/view?usp=sharing

# https://drive.google.com/file/d/1cOjcgQ8tZl6tOe9FQb05ZCFnUXy1MKwJ/view?usp=sharing

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1cOjcgQ8tZl6tOe9FQb05ZCFnUXy1MKwJ',
                                    dest_path='/content/gdg.zip',
                                    unzip=True)

Unzipping...Done.


In [2]:
# !pip install tensorflow==1.14.0
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [ ]:
print(tf.__version__)

1.14.0


In [36]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255., # rescaling
                                   rotation_range = 40,  # for augmentation
                                   width_shift_range = 0.2,
                                   validation_split = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.,validation_split = 0.2)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory("/content/gdg/train",
                                                    batch_size = 8,
                                                    subset="training",
                                                    class_mode = 'categorical', 
                                                    shuffle = True,
                                                    target_size = (100, 100))


test_generator = test_datagen.flow_from_directory("/content/gdg/test",
                                                    batch_size = 8,
                                                    class_mode = None, 
                                                    target_size = (100, 100))

val_generator = val_datagen.flow_from_directory("/content/gdg/train",
                                                subset = "validation",
                                                    batch_size = 8,
                                                    class_mode = 'categorical', 
                                                    target_size = (100, 100))

Found 471 images belonging to 3 classes.
Found 92 images belonging to 2 classes.
Found 117 images belonging to 3 classes.


In [4]:
import numpy as np
class_names=['lantana', 'parthenium', 'prosopis']
# test_img = np.array(test_generator)

In [13]:
# INPUT_SHAPE = (100, 100, 3)
# # get the VGG19 model
# vgg_layers = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, 
#                                                input_shape=INPUT_SHAPE)

# vgg_layers.summary()

In [12]:
# Fine-tune all the layers
# for layer in vgg_layers.layers:
#     layer.trainable = True

# # Check the trainable status of the individual layers
# for layer in vgg_layers.layers:
#     print(layer, layer.trainable)

In [17]:
# define sequential model
# model = tf.keras.models.Sequential()

# # Add the vgg convolutional base model
# model.add(vgg_layers)

# # add flatten layer
# model.add(tf.keras.layers.Flatten())

# # add dense layers with some dropout
# # model.add(tf.keras.layers.Dense(256, activation='relu'))
# # model.add(tf.keras.layers.Dropout(rate=0.3))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
# model.add(tf.keras.layers.Dropout(rate=0.1))

# model.add(tf.keras.layers.Dense(32, activation='relu'))
# model.add(tf.keras.layers.Dropout(rate=0.1))

# # add output layer
# model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [37]:
input_shape= (100,100,3)


# get the VGG19 model
vgg_layers = tf.keras.applications.vgg19.VGG19(weights='imagenet', include_top=False, 
                                               input_shape=INPUT_SHAPE)

vgg_layers.summary()
inputs = tf.keras.Input(shape=input_shape)

from tensorflow.keras.applications.inception_v3 import InceptionV3
# base_model = InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')

# for layer in base_model.layers:
#     layer.trainable = True

#Augmenting the images
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
    ]
)


model = Sequential()
model.add(data_augmentation)
# model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))
model.add(tf.keras.layers.Dense(10, input_shape=input_shape))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(3,activation='softmax'))

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [45]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [52]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, 
                                               restore_best_weights=True,
                                               verbose=1)

model.fit(train_generator,
          callbacks=[es_callback],
          validation_data = val_generator,
          epochs = 30)

Epoch 1/30
59/59 [==============================] - 2s 31ms/step - loss: 0.9752 - accuracy: 0.5180 - val_loss: 0.8839 - val_accuracy: 0.6154
Epoch 2/30
59/59 [==============================] - 2s 31ms/step - loss: 0.9826 - accuracy: 0.5202 - val_loss: 0.8834 - val_accuracy: 0.6239
Epoch 3/30
59/59 [==============================] - 2s 31ms/step - loss: 0.9838 - accuracy: 0.4862 - val_loss: 0.8807 - val_accuracy: 0.6239
Epoch 4/30
59/59 [==============================] - 2s 30ms/step - loss: 0.9945 - accuracy: 0.5032 - val_loss: 0.8842 - val_accuracy: 0.6239
Epoch 5/30
59/59 [==============================] - 2s 31ms/step - loss: 0.9764 - accuracy: 0.5180 - val_loss: 0.8807 - val_accuracy: 0.6410
Epoch 6/30
59/59 [==============================] - 2s 30ms/step - loss: 0.9770 - accuracy: 0.5180 - val_loss: 0.8829 - val_accuracy: 0.6325
Epoch 7/30
59/59 [==============================] - 2s 30ms/step - loss: 0.9485 - accuracy: 0.5287 - val_loss: 0.8853 - val_accuracy: 0.6154
Epoch 8/30
59

In [53]:
import pandas as pd

test_generator.reset()
preds = model.predict_generator(test_generator)


In [49]:
preds

array([[0.8466426 , 0.08258957, 0.07076772],
       [0.08029717, 0.14656287, 0.77313995],
       [0.3933465 , 0.32824084, 0.27841273],
       [0.28710732, 0.4981571 , 0.21473554],
       [0.65858716, 0.19699226, 0.14442053],
       [0.17517903, 0.27090397, 0.553917  ],
       [0.22776236, 0.32722962, 0.44500798],
       [0.25598517, 0.4196991 , 0.3243157 ],
       [0.32118   , 0.38290069, 0.2959193 ],
       [0.06552112, 0.10057934, 0.8338996 ],
       [0.22354527, 0.49504957, 0.28140512],
       [0.24446908, 0.42790234, 0.3276286 ],
       [0.33799738, 0.43231824, 0.22968437],
       [0.55658305, 0.2864968 , 0.15692021],
       [0.97544855, 0.01139818, 0.01315328],
       [0.7096765 , 0.19497752, 0.09534603],
       [0.8360467 , 0.08764447, 0.07630885],
       [0.42176622, 0.3539941 , 0.22423968],
       [0.3507519 , 0.35800546, 0.29124263],
       [0.68866694, 0.18087249, 0.13046055],
       [0.29144818, 0.3875313 , 0.3210206 ],
       [0.46445006, 0.33462292, 0.20092705],
       [0.

In [50]:
predicted_class_indices  = np.argmax(preds,axis=1)


In [ ]:
model.save('model.h5')

In [51]:
predicted_class_indices

array([0, 2, 0, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       2, 2, 0, 1, 1, 0, 0, 1, 2, 2, 0, 2, 0, 1, 1, 1, 2, 2, 1, 1, 2, 0,
       2, 2, 2, 0, 0, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0,
       1, 2, 0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0, 1, 2, 2, 0,
       1, 1, 1, 2])

In [ ]:
#if using softmax activation on output layer
predicted_class_indices  = np.argmax(preds,axis=1)

labels = (train_generator.class_indices)

In [ ]:
labels

{'lantana': 0, 'parthenium': 1, 'prosopis': 2}

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

     |████████████████████████████████| 71kB 3.4MB/s 


/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
